In [1]:
%matplotlib inline
from pathlib import Path
import pandas as pd

from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [2]:
data_path = Path('../data/')
df        = pd.read_csv(data_path / 'interim' / 'fed_emp.csv')

In [3]:
# Removing the lines for which the target in unknown
df                                   = df[~df.EDLVL.isnull()]
# Removing the nan values in columns by either adding a new category
# or dropping the lines
df.loc[df.GSEGRD.isnull(), 'GSEGRD'] = 0
df.loc[df.OCC.isnull(), 'OCC']       = 0
df                                   = df[~df.SUPERVIS.isnull()]
df                                   = df[~df.TOA.isnull()]
df                                   = df[~df.SALARY.isnull()]
df                                   = df[~df.LOS.isnull()]
# df.isnull().sum(axis = 0)

In [4]:
df_data    = df.drop(['EDLVL'], axis = 1)
df_target  = df['EDLVL']
df_target  = df_target - 1 # Values between 0 and 21 instead of 1 and 22
# df_target.hist(bins = 22)

In [5]:
numerical_columns = ['SALARY', 'LOS']
df_num            = df_data[numerical_columns]
num_val_mean      = df_num.mean(axis = 0)
num_val_std       = df_num.std(axis = 0)
df_num            = (df_num - num_val_mean) / num_val_std
df_cat            = df_data.drop(numerical_columns, axis = 1) 

In [6]:
columns_encoders = {
    col : {
        val : i 
        for i, val in enumerate(df[col].unique())
    }
    for col in df_cat.columns
}
column_order = list(columns_encoders.keys())

In [7]:
for col in df_cat.columns:
    df_cat[col] = df_cat[col].apply(lambda x: columns_encoders[col][x])

In [8]:
class CategoricalEmbeddings(nn.Module):
    def __init__(self, col_order, col_encoders, col_to_emb_dim):
        super(CategoricalEmbeddings, self).__init__()
        self.col_order = col_order 
        self.cat_embs  = nn.ModuleDict({
            col: nn.Embedding(len(col_encoders[col]), col_to_emb_dim[col])
            for col in col_order
        })
        
    def forward(self, cat_variables):
        embeddings = [self.cat_embs[col](cat_variables[col]) for col in self.col_order]
        
        return torch.cat(embeddings, dim = 1)

In [9]:
class EdlvlClassifier(nn.Module):
    def __init__(self, col_order, col_encoders, col_to_emb_dim):
        super(EdlvlClassifier, self).__init__()
        self.cat_emb    = CategoricalEmbeddings(col_order, col_encoders, col_to_emb_dim)
        sum_cat_emb_dim = sum(col_to_emb_dim.values())
        self.linear1    = nn.Linear(sum_cat_emb_dim + 2, 128)
        self.linear2    = nn.Linear(128, 22)
        
    def forward(self, cat_variables, num_variables):
#         import pdb
#         pdb.set_trace()
        cat_embeddings = self.cat_emb(cat_variables)
        cat_num_tensor = torch.cat([cat_embeddings, num_variables], dim = 1)
        out_linear1    = F.relu(self.linear1(cat_num_tensor))
        out_linear2    = self.linear2(out_linear1)
        
        return out_linear2

In [10]:
model = EdlvlClassifier(
    column_order,
    columns_encoders,
    {
        col : 2
        for col in columns_encoders
    }
)
model

EdlvlClassifier(
  (cat_emb): CategoricalEmbeddings(
    (cat_embs): ModuleDict(
      (AGELVL): Embedding(12, 2)
      (AGYSUB): Embedding(523, 2)
      (GSEGRD): Embedding(16, 2)
      (LOC): Embedding(219, 2)
      (LOSLVL): Embedding(10, 2)
      (OCC): Embedding(656, 2)
      (PATCO): Embedding(7, 2)
      (PPGRD): Embedding(933, 2)
      (SALLVL): Embedding(25, 2)
      (STEMOCC): Embedding(100, 2)
      (SUPERVIS): Embedding(6, 2)
      (TOA): Embedding(18, 2)
      (WORKSCH): Embedding(10, 2)
      (WORKSTAT): Embedding(2, 2)
    )
  )
  (linear1): Linear(in_features=30, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=22, bias=True)
)

In [11]:
dataset = TensorDataset(
    *[
        torch.tensor(df_cat[col].values)
        for col in column_order
    ], # categorical variables in the correct order
    torch.tensor(df_num.values, dtype = torch.float32), # numerical variables
    torch.tensor(df_target.values, dtype = torch.int64) # target variables
)

In [12]:
loader = DataLoader(dataset, batch_size = 16, shuffle = False)

In [13]:
device     = torch.device('cuda')
model      = model.to(device)
epochs     = 20
batch_size = 2048
optimizer  = optim.Adam(model.parameters())
criterion  = nn.CrossEntropyLoss()
loader     = DataLoader(dataset, batch_size = batch_size, shuffle = False)

In [14]:
for epoch in range(epochs):
    correct = 0
    total   = 0
    for i, (*cat_var_list, num_var, y) in enumerate(loader):
        optimizer.zero_grad()
        cat_var_list   = [t.to(device) for t in cat_var_list]
        num_var        = num_var.to(device)
        y              = y.to(device)
        cat_variables  = dict(zip(column_order, cat_var_list))
        res            = model(cat_variables, num_var)
        loss           = criterion(res, y)
        correct       += (res.argmax(dim = 1) == y).detach().sum()
        total         += y.shape[0]
        loss.backward()
        optimizer.step()
        if i % 300 == 0: 
            print(f'[{epoch}:{i}] {100 * correct / total}%, {loss.item()}')

[0:0] 1%, 3.1532175540924072
[0:300] 40%, 1.2752782106399536
[0:600] 40%, 1.8239728212356567
[1:0] 40%, 1.7195374965667725
[1:300] 46%, 1.115164041519165
[1:600] 44%, 1.79296875
[2:0] 38%, 1.6253256797790527
[2:300] 47%, 1.0749456882476807
[2:600] 45%, 1.7649394273757935
[3:0] 39%, 1.5822863578796387
[3:300] 47%, 1.0605298280715942
[3:600] 46%, 1.7483768463134766
[4:0] 40%, 1.5597341060638428
[4:300] 48%, 1.0514476299285889
[4:600] 46%, 1.7341982126235962
[5:0] 41%, 1.5508431196212769
[5:300] 48%, 1.0451009273529053
[5:600] 47%, 1.7198361158370972
[6:0] 41%, 1.5447018146514893
[6:300] 49%, 1.0388576984405518
[6:600] 47%, 1.7047725915908813
[7:0] 41%, 1.5358941555023193
[7:300] 49%, 1.0325647592544556
[7:600] 47%, 1.6919487714767456
[8:0] 41%, 1.530129313468933
[8:300] 49%, 1.0289430618286133
[8:600] 47%, 1.680229663848877
[9:0] 41%, 1.524391531944275
[9:300] 49%, 1.025395393371582
[9:600] 48%, 1.6679965257644653
[10:0] 41%, 1.5193424224853516
[10:300] 49%, 1.0231181383132935
[10:600] 4